# Lightly - 5%, 7.5%, 10%, 12.5%, 15%

In [38]:
from os import linesep
from pathlib import Path
from datetime import datetime
import platform 
from lightly.api import ApiWorkflowClient
from lightly.openapi_generated.swagger_client import DatasetType, DatasourcePurpose
import torch
import time
from typing import List

In [39]:
LIGHTLY_TOKEN = "6ef4b5e20f6a1dba87a72a9eb4ddceb3f9529cd3d46b94a8" 
# DATASET_PATH = Path("FOCAL/yolov5_format/images/train")
DATASET_PATH = Path("/home/avgus/active-vehicle-detection/clothing_dataset")  # e.g., Path("/path/to/images") or Path("clothing_dataset")
DATASET_NAME = "Teste_proportion2"

In [40]:
def monitoring_runs(client: ApiWorkflowClient, run_ids: List[str]):
    """
    Monitora múltiplos runs de workers no Lightly de forma concorrente.
    Args:
        client: A instância do cliente da API do Lightly.
        run_ids: Uma lista de strings, onde cada string é um ID de run agendado.
    """
    if not run_ids:
        print("⚠️ Nenhuma ID de run foi fornecida.")
        return {}
    n_ids = len(run_ids)

    print(f"🚀 Iniciando monitoramento para {n_ids} Ciclos de Active Learning...")
    i = 0
    for run_id in run_ids:
        i += 1
        print(f"\n🔍 Monitorando Ciclo {i}/{n_ids}:")
        print(f"  -> ID [{run_id}]")

        for run_info in client.compute_worker_run_info_generator(scheduled_run_id=run_id):
            print(f"Estado Atual: '{run_info.state}' ---> '{run_info.message}'")

        if run_info.ended_successfully():
            print(f"✅ Run [{run_id}] concluído com SUCESSO!")
        else:
            print(f"❌ Run [{run_id}] FALHOU com status final: '{run_info.state}'")
    

    print("\n🏁 Monitoramento concluído para todos os runs.")

Iniciar Cliente Lightly

In [41]:
### Cliente Ligthly ###
client = ApiWorkflowClient(token=LIGHTLY_TOKEN)

Criar Dataset

In [42]:
client.create_dataset(
    dataset_name=DATASET_NAME,
    dataset_type=DatasetType.IMAGES,
)

client.set_local_config(purpose=DatasourcePurpose.INPUT)
client.set_local_config(purpose=DatasourcePurpose.LIGHTLY)
print(f"dataset_id: {client.dataset_id}")

dataset_id: 6877f8768388df88c81c54e2


In [43]:
client.set_local_config(purpose=DatasourcePurpose.INPUT)
client.set_local_config(purpose=DatasourcePurpose.LIGHTLY)

In [44]:
# Let's fetch the dataset we created above.
# client.set_dataset_id_by_name(dataset_name=DATASET_NAME)

#### Agendar uma seleção

In [45]:
run_ids = []

Ciclo 1 - warm up - 1% dos dados escolhidos aleatoriamente

In [ ]:
# Schedule the run.
scheduled_run_id = client.schedule_compute_worker_run(
    worker_config={
        # "shutdown_when_job_finished": True,
        "use_datapool": True,
        "datasource": {
            "process_all": True,
        },
    },
    selection_config={
        "proportion_samples": 0.01, # 1% do dataset
        "strategies": [
            {
                "input": {
                    "type": "RANDOM",
                    "random_seed": 42, # optional, for reproducibility
                },
                "strategy": {
                    "type": "WEIGHTS",
                }
            }
        ]
    },
)
print(f"scheduled_run_id: {scheduled_run_id}")
run_ids.append(scheduled_run_id)

scheduled_run_id: 6877f87d47b73983f4c36606


Ciclo 2 - +1% dos dados => 2% do total

In [ ]:
# Schedule the run.
scheduled_run_id = client.schedule_compute_worker_run(
    worker_config={
        # "shutdown_when_job_finished": True,
        "use_datapool": True,
        "datasource": {
            "process_all": True,
        },
    },
    selection_config={
        "proportion_samples": 0.01, # +1% do dataset
        "strategies": [
            {
                "input": {
                    "type": "EMBEDDINGS",
                },
                "strategy": {
                    "type": "DIVERSITY",
                },
            }
        ],
    },
)
print(f"scheduled_run_id: {scheduled_run_id}")
run_ids.append(scheduled_run_id)

scheduled_run_id: 6877f87f8388df88c81c54e5


Ciclo 3 - +2% dos dados => 4% do total

In [ ]:
# Schedule the run.
scheduled_run_id = client.schedule_compute_worker_run(
    worker_config={
        # "shutdown_when_job_finished": True,
        "use_datapool": True,
        "datasource": {
            "process_all": True,
        },
    },
    selection_config={
        "proportion_samples": 0.02, # +2% do dataset
        "strategies": [
            {
                "input": {
                    "type": "EMBEDDINGS",
                },
                "strategy": {
                    "type": "DIVERSITY",
                },
            }
        ],
    },
)
print(f"scheduled_run_id: {scheduled_run_id}")
run_ids.append(scheduled_run_id)

scheduled_run_id: 6877f8808388df88c81c54e6


Ciclo 4 - +4% dos dados => 8% do total

In [ ]:
# Schedule the run.
scheduled_run_id = client.schedule_compute_worker_run(
    worker_config={
        # "shutdown_when_job_finished": True,
        "use_datapool": True,
        "datasource": {
            "process_all": True,
        },
    },
    selection_config={
        "proportion_samples": 0.04, # +4% do dataset
        "strategies": [
            {
                "input": {
                    "type": "EMBEDDINGS",
                },
                "strategy": {
                    "type": "DIVERSITY",
                },
            }
        ],
    },
)
print(f"scheduled_run_id: {scheduled_run_id}")
run_ids.append(scheduled_run_id)

scheduled_run_id: 6877f8828388df88c81c54e7


Ciclo 5 - +8% dos dados => 16% do total

In [ ]:
# Schedule the run.
scheduled_run_id = client.schedule_compute_worker_run(
    worker_config={
        # "shutdown_when_job_finished": True,
        "use_datapool": True,
        "datasource": {
            "process_all": True,
        },
    },
    selection_config={
        "proportion_samples": 0.08, # +8% do dataset
        "strategies": [
            {
                "input": {
                    "type": "EMBEDDINGS",
                },
                "strategy": {
                    "type": "DIVERSITY",
                },
            }
        ],
    },
)
print(f"scheduled_run_id: {scheduled_run_id}")
run_ids.append(scheduled_run_id)

scheduled_run_id: 6877f88447b73983f4c3660a


In [51]:
# absolute_dataset_path = Path(DATASET_PATH).absolute()
# absolute_lightly_path = Path("lightly").absolute()

Comandos Docker

In [52]:
print(
    f"{linesep}Docker Run command: {linesep}"
    f"\033[7m"
    f"docker run --shm-size='1024m' --rm -it \\{linesep}"
    f"\t-v '{DATASET_PATH.absolute()}':/input_mount:ro \\{linesep}"
    f"\t-v '{Path('lightly').absolute()}':/lightly_mount \\{linesep}"
    f"\t-e LIGHTLY_TOKEN={LIGHTLY_TOKEN} \\{linesep}"
    f"\tlightly/worker:latest{linesep}"
    f"\033[0m"
)
print(
    f"{linesep}Lightly Serve command:{linesep}"
    f"\033[7m"
    f"lightly-serve \\{linesep}"
    f"\tinput_mount='{DATASET_PATH.absolute()}' \\{linesep}"
    f"\tlightly_mount='{Path('lightly').absolute()}'{linesep}"
    f"\033[0m"
)


Docker Run command: 
docker run --shm-size='1024m' --rm -it \
	-v '/home/avgus/active-vehicle-detection/clothing_dataset':/input_mount:ro \
	-v '/home/avgus/active-vehicle-detection/src/lightly':/lightly_mount \
	-e LIGHTLY_TOKEN=6ef4b5e20f6a1dba87a72a9eb4ddceb3f9529cd3d46b94a8 \
	lightly/worker:latest


Lightly Serve command:
lightly-serve \
	input_mount='/home/avgus/active-vehicle-detection/clothing_dataset' \
	lightly_mount='/home/avgus/active-vehicle-detection/src/lightly'



Monitorando as seleções

In [53]:
monitoring_runs(client, run_ids)

🚀 Iniciando monitoramento para 5 Ciclos de Active Learning...

🔍 Monitorando Ciclo 1/5:
  -> ID [6877f87d47b73983f4c36606]
Estado Atual: 'OPEN' ---> 'Waiting for pickup by Lightly Worker. Make sure to start a Lightly Worker connected to your user token to process the job.'
Estado Atual: 'EMBEDDING' ---> 'State set to EMBEDDING'
Estado Atual: 'GENERATING_REPORT' ---> 'State set to GENERATING_REPORT'
Estado Atual: 'COMPLETED' ---> 'State set to COMPLETED'
✅ Run [6877f87d47b73983f4c36606] concluído com SUCESSO!

🔍 Monitorando Ciclo 2/5:
  -> ID [6877f87f8388df88c81c54e5]
Estado Atual: 'OPEN' ---> 'Waiting for pickup by Lightly Worker. Make sure to start a Lightly Worker connected to your user token to process the job.'
Estado Atual: 'EMBEDDING' ---> 'State set to EMBEDDING'
Estado Atual: 'UPLOADING_ARTIFACTS' ---> 'State set to UPLOADING_ARTIFACTS'
Estado Atual: 'COMPLETED' ---> 'State set to COMPLETED'
✅ Run [6877f87f8388df88c81c54e5] concluído com SUCESSO!

🔍 Monitorando Ciclo 3/5:
  ->